In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0   conda-forge
    geopy:         1.18.1-py_0 conda-forge

geographiclib- 100% |################################| Time: 0:00:00  23.94 MB/s
geopy-1.18.1-p 100% |################################| Time: 0:00:00  37.87 MB/s
Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    altair:  2.2.2-py35_1 conda-forge
    branca:  0.3.1-py_0   conda-forge
    folium:  0.5.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forge

altair-2.2.2-p 100% |################################| Time: 0:00:00  56.78 MB/s
branca-0.3.1-p 100% |################################| Time: 0:00:00  35.94 MB/s
vincent-0.4.4- 100% |###################

In [2]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')

Data downloaded!


In [3]:
# path of data 
path = 'https://data.cityofnewyork.us/api/views/swpk-hqdp/rows.csv?accessType=DOWNLOAD'
df = pd.read_csv(path)
df.head()




,Borough,Year,FIPS County Code,NTA Code,NTA Name,Population
0,Bronx,2000,5,BX01,Claremont-Bathgate,28149
1,Bronx,2000,5,BX03,Eastchester-Edenwald-Baychester,35422
2,Bronx,2000,5,BX05,Bedford Park-Fordham North,55329
3,Bronx,2000,5,BX06,Belmont,25967
4,Bronx,2000,5,BX07,Bronxdale,34309


In [4]:
df = df.loc[df['Borough'] == 'Manhattan']
df = df.loc[df['Year'] == 2010]
df.head()

,Borough,Year,FIPS County Code,NTA Code,NTA Name,Population
284,Manhattan,2010,61,MN01,Marble Hill-Inwood,46746
285,Manhattan,2010,61,MN03,Central Harlem North-Polo Grounds,75282
286,Manhattan,2010,61,MN04,Hamilton Heights,48520
288,Manhattan,2010,61,MN06,Manhattanville,22950
289,Manhattan,2010,61,MN09,Morningside Heights,55929


In [5]:
df.sort_values(by = 'NTA Name', axis = 0, inplace = True)
df.head(20)

,Borough,Year,FIPS County Code,NTA Code,NTA Name,Population
302,Manhattan,2010,61,MN25,Battery Park City-Lower Manhattan,39699
285,Manhattan,2010,61,MN03,Central Harlem North-Polo Grounds,75282
290,Manhattan,2010,61,MN11,Central Harlem South,43383
303,Manhattan,2010,61,MN27,Chinatown,47844
294,Manhattan,2010,61,MN15,Clinton,45884
308,Manhattan,2010,61,MN34,East Harlem North,58019
307,Manhattan,2010,61,MN33,East Harlem South,57902
299,Manhattan,2010,61,MN22,East Village,44136
298,Manhattan,2010,61,MN21,Gramercy,27988
286,Manhattan,2010,61,MN04,Hamilton Heights,48520


In [6]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

In [7]:
neighborhoods_data = newyork_data['features']

In [8]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

In [9]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)
    
    

In [10]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [11]:
manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
manhattan_data.sort_values(by = 'Neighborhood', axis = 0, inplace = True)

manhattan_data.shape

(40, 4)

In [12]:
CLIENT_ID = 'BXZPG3NUBOSGVJMFMDGMLMD1SYCNOTRGH5ZMAA1OY0C3HJ50' # your Foursquare ID
CLIENT_SECRET = 'YQFFK1MY1VF4MRKLHXZAP03VSXT1UNCYHXUKTR1H4NUPNDU2' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: BXZPG3NUBOSGVJMFMDGMLMD1SYCNOTRGH5ZMAA1OY0C3HJ50
CLIENT_SECRET:YQFFK1MY1VF4MRKLHXZAP03VSXT1UNCYHXUKTR1H4NUPNDU2


In [13]:
address = 'Manhattan, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Manhattan are 40.7900869, -73.9598295.


## Search for gyms in Manhattan area

In [14]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        LIMIT = 100 # limit of number of venues returned by Foursquare API
        radius = 500 # define radiu
        search_query = 'Gym','yoga'
        url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            lat, 
            lng,
            VERSION,
            search_query,
            radius, 
            LIMIT)

        results = requests.get(url).json()["response"]['venues']
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['name'],
            v['location']['formattedAddress'][-2],
            v['location']['distance'],
            v['location']['lat'], 
            v['location']['lng'],
            ) for v in results])

    nearby_venues = pd.DataFrame([item for gym_venues in venues_list for item in gym_venues])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'PostalCode',
                  'distance',
                  'Venue Latitude', 
                  'Venue Longitude',
                  ]      
    
    return(nearby_venues)


## Define corresponding url

In [49]:
# type your answer here

manhattan_venues = getNearbyVenues(names=manhattan_data['Neighborhood'],
                                   latitudes=manhattan_data['Latitude'],
                                   longitudes=manhattan_data['Longitude']
                                  )


Battery Park City
Carnegie Hill
Central Harlem
Chelsea
Chinatown
Civic Center
Clinton
East Harlem
East Village
Financial District
Flatiron
Gramercy
Greenwich Village
Hamilton Heights
Hudson Yards
Inwood
Lenox Hill
Lincoln Square
Little Italy
Lower East Side
Manhattan Valley
Manhattanville
Marble Hill
Midtown
Midtown South
Morningside Heights
Murray Hill
Noho
Roosevelt Island
Soho
Stuyvesant Town
Sutton Place
Tribeca
Tudor City
Turtle Bay
Upper East Side
Upper West Side
Washington Heights
West Village
Yorkville


In [50]:
print(manhattan_venues.shape)
manhattan_venues.head(5)

(1203, 8)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,PostalCode,distance,Venue Latitude,Venue Longitude
0,Battery Park City,40.711932,-74.016869,225 Rector Place Gym and Studio,"New York, NY 10280",328,40.709183,-74.015441
1,Battery Park City,40.711932,-74.016869,Gym @ The Terrace Club,"New York, NY 10280",350,40.708995,-74.018358
2,Battery Park City,40.711932,-74.016869,The Gym At Soundings,"New York, NY 10280",368,40.708625,-74.016795
3,Battery Park City,40.711932,-74.016869,The Gym @ The Riverhouse,"New York, NY 10282",380,40.715301,-74.016120
4,Battery Park City,40.711932,-74.016869,Stuyvesant High School (3rd floor gym),"New York, NY 10282",196,40.710186,-74.016563


In [52]:
manhattan_venues['PostalCode'] = manhattan_venues['PostalCode'].str.replace("New York, NY", " ")
manhattan_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,PostalCode,distance,Venue Latitude,Venue Longitude
0,Battery Park City,40.711932,-74.016869,225 Rector Place Gym and Studio,10280,328,40.709183,-74.015441
1,Battery Park City,40.711932,-74.016869,Gym @ The Terrace Club,10280,350,40.708995,-74.018358
2,Battery Park City,40.711932,-74.016869,The Gym At Soundings,10280,368,40.708625,-74.016795
3,Battery Park City,40.711932,-74.016869,The Gym @ The Riverhouse,10282,380,40.715301,-74.016120
4,Battery Park City,40.711932,-74.016869,Stuyvesant High School (3rd floor gym),10282,196,40.710186,-74.016563


In [17]:
manhattan_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,PostalCode,distance,Venue Latitude,Venue Longitude
Neighborhood,,,,,,,
Battery Park City,41,41,41,41,41,41,41
Carnegie Hill,36,36,36,36,36,36,36
Central Harlem,10,10,10,10,10,10,10
Chelsea,39,39,39,39,39,39,39
Chinatown,18,18,18,18,18,18,18
Civic Center,50,50,50,50,50,50,50
Clinton,50,50,50,50,50,50,50
East Harlem,8,8,8,8,8,8,8
East Village,28,28,28,28,28,28,28


In [40]:
print('There are {} fitness centers in the neighborhood of Manhattan.'.format(len(manhattan_venues['Venue'].unique())))

There are 925 fitness centers in the neighborhood of Manhattan.


In [29]:
search_query = 'Gym'
radius = 10000
LIMIT = 100
print(search_query + ' .... OK!')

Gym .... OK!


In [30]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=BXZPG3NUBOSGVJMFMDGMLMD1SYCNOTRGH5ZMAA1OY0C3HJ50&client_secret=YQFFK1MY1VF4MRKLHXZAP03VSXT1UNCYHXUKTR1H4NUPNDU2&ll=40.7900869,-73.9598295&v=20180605&query=Gym&radius=10000&limit=100'

In [31]:
results = requests.get(url).json()
results
# assign relevant part of JSON to venues
gym_venues = results['response']['venues']

# tranform venues into a dataframe
gym_df = json_normalize(yoga_venues)
gym_df.head()

,categories,delivery.id,delivery.provider.icon.name,delivery.provider.icon.prefix,delivery.provider.icon.sizes,delivery.provider.name,delivery.url,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'shortName': 'Yoga Studio', 'id': '4bf58dd8d...",NaN,NaN,NaN,NaN,NaN,NaN,False,4a049afcf964a5200f721fe3,203 E 86th St,US,New York,United States,at 3rd Ave,1364,"[203 E 86th St (at 3rd Ave), New York, NY 1002...","[{'label': 'display', 'lat': 40.77872347864471...",40.778723,-73.953772,NaN,10028,NY,Pure Yoga East,v-1549822849,NaN
1,"[{'shortName': 'Yoga Studio', 'id': '4bf58dd8d...",344817,/delivery_provider_seamless_20180129.png,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",seamless,https://www.seamless.com/menu/motorino-upper-w...,False,516827db72da218d58fdf80b,75 W 85th St Apt B,US,New York,United States,btwn Columbus Ave. & Central Park West,1160,[75 W 85th St Apt B (btwn Columbus Ave. & Cent...,"[{'label': 'display', 'lat': 40.78568220087445...",40.785682,-73.972310,NaN,10024,NY,Upper West Side Yoga and Wellness,v-1549822849,53981514
2,"[{'shortName': 'Yoga Studio', 'id': '4bf58dd8d...",NaN,NaN,NaN,NaN,NaN,NaN,False,4b365ac3f964a520773325e3,204 W 77th St,US,New York,United States,at Amsterdam Ave,1908,"[204 W 77th St (at Amsterdam Ave), New York, N...","[{'label': 'display', 'lat': 40.7816589375539,...",40.781659,-73.979545,NaN,10024,NY,Pure Yoga West,v-1549822849,NaN
3,"[{'shortName': 'Yoga Studio', 'id': '4bf58dd8d...",NaN,NaN,NaN,NaN,NaN,NaN,False,4ad7bdb0f964a5204d0e21e3,132 E 85th St,US,New York,United States,Lexington Ave,1282,"[132 E 85th St (Lexington Ave), New York, NY 1...","[{'label': 'display', 'lat': 40.77890950875221...",40.778910,-73.956160,NaN,10028,NY,New York Yoga - Hot Studio,v-1549822849,48664255
4,"[{'shortName': 'Yoga Studio', 'id': '4bf58dd8d...",NaN,NaN,NaN,NaN,NaN,NaN,False,4a86b49cf964a5201d0120e3,2744 Broadway,US,New York,United States,@ 105th Street,1345,"[2744 Broadway (@ 105th Street), New York, NY ...","[{'label': 'display', 'lat': 40.80051468505158...",40.800515,-73.967910,NaN,10025,NY,Life in Motion: Yoga Cycling Barre,v-1549822849,33184348


In [32]:
# keep only columns that include venue name, and anything that is associated with location
gym_filtered_columns = ['name', 'categories'] + [col for col in gym_df.columns if col.startswith('location.')] + ['id']
gym_df_filtered = gym_df.loc[:, gym_filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
gym_df_filtered['categories'] = gym_df_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
gym_df_filtered.columns = [column.split('.')[-1] for column in gym_df_filtered.columns]

gym_df_filtered


,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Pure Yoga East,Yoga Studio,203 E 86th St,US,New York,United States,at 3rd Ave,1364,"[203 E 86th St (at 3rd Ave), New York, NY 1002...","[{'label': 'display', 'lat': 40.77872347864471...",40.778723,-73.953772,NaN,10028,NY,4a049afcf964a5200f721fe3
1,Upper West Side Yoga and Wellness,Yoga Studio,75 W 85th St Apt B,US,New York,United States,btwn Columbus Ave. & Central Park West,1160,[75 W 85th St Apt B (btwn Columbus Ave. & Cent...,"[{'label': 'display', 'lat': 40.78568220087445...",40.785682,-73.972310,NaN,10024,NY,516827db72da218d58fdf80b
2,Pure Yoga West,Yoga Studio,204 W 77th St,US,New York,United States,at Amsterdam Ave,1908,"[204 W 77th St (at Amsterdam Ave), New York, N...","[{'label': 'display', 'lat': 40.7816589375539,...",40.781659,-73.979545,NaN,10024,NY,4b365ac3f964a520773325e3
3,New York Yoga - Hot Studio,Yoga Studio,132 E 85th St,US,New York,United States,Lexington Ave,1282,"[132 E 85th St (Lexington Ave), New York, NY 1...","[{'label': 'display', 'lat': 40.77890950875221...",40.778910,-73.956160,NaN,10028,NY,4ad7bdb0f964a5204d0e21e3
4,Life in Motion: Yoga Cycling Barre,Yoga Studio,2744 Broadway,US,New York,United States,@ 105th Street,1345,"[2744 Broadway (@ 105th Street), New York, NY ...","[{'label': 'display', 'lat': 40.80051468505158...",40.800515,-73.967910,NaN,10025,NY,4a86b49cf964a5201d0120e3
5,NY Loves Yoga,Yoga Studio,140 West 83rd Street - Garden Level,US,New York,United States,Amsterdam and Columbus Ave,1402,[140 West 83rd Street - Garden Level (Amsterda...,"[{'label': 'display', 'lat': 40.78520461497391...",40.785205,-73.975170,NaN,10024,NY,4b55d36ff964a520b3f127e3
6,Harlem Yoga Studio,Yoga Studio,44 W 125th St Fl 3,US,New York,United States,btw Lenox and 5th,2313,"[44 W 125th St Fl 3 (btw Lenox and 5th), New Y...","[{'label': 'display', 'lat': 40.80707697816692...",40.807077,-73.944010,NaN,10027,NY,4d48fce9a0ef548126b5edf6
7,Modo Yoga NYC,Yoga Studio,434 Avenue of the Americas,US,New York,United States,10th Street,6983,"[434 Avenue of the Americas (10th Street), New...","[{'label': 'display', 'lat': 40.73467444734881...",40.734674,-73.998670,NaN,10011,NY,4f2425cce4b0e12081504088
8,Spring Into Yoga- Central Park,Yoga Studio,102nd St and Fifth Avenue,US,New York,United States,"102nd st, Fifth Ave",567,"[102nd St and Fifth Avenue (102nd st, Fifth Av...","[{'label': 'display', 'lat': 40.79225934919049...",40.792259,-73.953744,NaN,10029,NY,51bd486babd8c63049739fd8
9,Yoga to the People,Yoga Studio,1017 Avenue of the Americas,US,New York,United States,38th St,4743,"[1017 Avenue of the Americas (38th St), New Yo...","[{'label': 'display', 'lat': 40.7523979170956,...",40.752398,-73.986091,NaN,10018,NY,4b4ce47cf964a5203bc326e3


In [25]:
search_query = 'yoga'
radius = 10000
LIMIT = 100
print(search_query + ' .... OK!')

yoga .... OK!


In [26]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=BXZPG3NUBOSGVJMFMDGMLMD1SYCNOTRGH5ZMAA1OY0C3HJ50&client_secret=YQFFK1MY1VF4MRKLHXZAP03VSXT1UNCYHXUKTR1H4NUPNDU2&ll=40.7900869,-73.9598295&v=20180605&query=yoga&radius=10000&limit=100'

In [27]:
results = requests.get(url).json()
results
# assign relevant part of JSON to venues
yoga_venues = results['response']['venues']

# tranform venues into a dataframe
yoga_df = json_normalize(yoga_venues)
yoga_df.head()

,categories,delivery.id,delivery.provider.icon.name,delivery.provider.icon.prefix,delivery.provider.icon.sizes,delivery.provider.name,delivery.url,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'shortName': 'Yoga Studio', 'id': '4bf58dd8d...",NaN,NaN,NaN,NaN,NaN,NaN,False,4a049afcf964a5200f721fe3,203 E 86th St,US,New York,United States,at 3rd Ave,1364,"[203 E 86th St (at 3rd Ave), New York, NY 1002...","[{'label': 'display', 'lat': 40.77872347864471...",40.778723,-73.953772,NaN,10028,NY,Pure Yoga East,v-1549822849,NaN
1,"[{'shortName': 'Yoga Studio', 'id': '4bf58dd8d...",344817,/delivery_provider_seamless_20180129.png,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",seamless,https://www.seamless.com/menu/motorino-upper-w...,False,516827db72da218d58fdf80b,75 W 85th St Apt B,US,New York,United States,btwn Columbus Ave. & Central Park West,1160,[75 W 85th St Apt B (btwn Columbus Ave. & Cent...,"[{'label': 'display', 'lat': 40.78568220087445...",40.785682,-73.972310,NaN,10024,NY,Upper West Side Yoga and Wellness,v-1549822849,53981514
2,"[{'shortName': 'Yoga Studio', 'id': '4bf58dd8d...",NaN,NaN,NaN,NaN,NaN,NaN,False,4b365ac3f964a520773325e3,204 W 77th St,US,New York,United States,at Amsterdam Ave,1908,"[204 W 77th St (at Amsterdam Ave), New York, N...","[{'label': 'display', 'lat': 40.7816589375539,...",40.781659,-73.979545,NaN,10024,NY,Pure Yoga West,v-1549822849,NaN
3,"[{'shortName': 'Yoga Studio', 'id': '4bf58dd8d...",NaN,NaN,NaN,NaN,NaN,NaN,False,4ad7bdb0f964a5204d0e21e3,132 E 85th St,US,New York,United States,Lexington Ave,1282,"[132 E 85th St (Lexington Ave), New York, NY 1...","[{'label': 'display', 'lat': 40.77890950875221...",40.778910,-73.956160,NaN,10028,NY,New York Yoga - Hot Studio,v-1549822849,48664255
4,"[{'shortName': 'Yoga Studio', 'id': '4bf58dd8d...",NaN,NaN,NaN,NaN,NaN,NaN,False,4a86b49cf964a5201d0120e3,2744 Broadway,US,New York,United States,@ 105th Street,1345,"[2744 Broadway (@ 105th Street), New York, NY ...","[{'label': 'display', 'lat': 40.80051468505158...",40.800515,-73.967910,NaN,10025,NY,Life in Motion: Yoga Cycling Barre,v-1549822849,33184348


In [28]:
# keep only columns that include venue name, and anything that is associated with location
yoga_filtered_columns = ['name', 'categories'] + [col for col in yoga_df.columns if col.startswith('location.')] + ['id']
yoga_df_filtered = yoga_df.loc[:, yoga_filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
yoga_df_filtered['categories'] = yoga_df_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
yoga_df_filtered.columns = [column.split('.')[-1] for column in yoga_df_filtered.columns]

yoga_df_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Pure Yoga East,Yoga Studio,203 E 86th St,US,New York,United States,at 3rd Ave,1364,"[203 E 86th St (at 3rd Ave), New York, NY 1002...","[{'label': 'display', 'lat': 40.77872347864471...",40.778723,-73.953772,NaN,10028,NY,4a049afcf964a5200f721fe3
1,Upper West Side Yoga and Wellness,Yoga Studio,75 W 85th St Apt B,US,New York,United States,btwn Columbus Ave. & Central Park West,1160,[75 W 85th St Apt B (btwn Columbus Ave. & Cent...,"[{'label': 'display', 'lat': 40.78568220087445...",40.785682,-73.972310,NaN,10024,NY,516827db72da218d58fdf80b
2,Pure Yoga West,Yoga Studio,204 W 77th St,US,New York,United States,at Amsterdam Ave,1908,"[204 W 77th St (at Amsterdam Ave), New York, N...","[{'label': 'display', 'lat': 40.7816589375539,...",40.781659,-73.979545,NaN,10024,NY,4b365ac3f964a520773325e3
3,New York Yoga - Hot Studio,Yoga Studio,132 E 85th St,US,New York,United States,Lexington Ave,1282,"[132 E 85th St (Lexington Ave), New York, NY 1...","[{'label': 'display', 'lat': 40.77890950875221...",40.778910,-73.956160,NaN,10028,NY,4ad7bdb0f964a5204d0e21e3
4,Life in Motion: Yoga Cycling Barre,Yoga Studio,2744 Broadway,US,New York,United States,@ 105th Street,1345,"[2744 Broadway (@ 105th Street), New York, NY ...","[{'label': 'display', 'lat': 40.80051468505158...",40.800515,-73.967910,NaN,10025,NY,4a86b49cf964a5201d0120e3
5,NY Loves Yoga,Yoga Studio,140 West 83rd Street - Garden Level,US,New York,United States,Amsterdam and Columbus Ave,1402,[140 West 83rd Street - Garden Level (Amsterda...,"[{'label': 'display', 'lat': 40.78520461497391...",40.785205,-73.975170,NaN,10024,NY,4b55d36ff964a520b3f127e3
6,Harlem Yoga Studio,Yoga Studio,44 W 125th St Fl 3,US,New York,United States,btw Lenox and 5th,2313,"[44 W 125th St Fl 3 (btw Lenox and 5th), New Y...","[{'label': 'display', 'lat': 40.80707697816692...",40.807077,-73.944010,NaN,10027,NY,4d48fce9a0ef548126b5edf6
7,Modo Yoga NYC,Yoga Studio,434 Avenue of the Americas,US,New York,United States,10th Street,6983,"[434 Avenue of the Americas (10th Street), New...","[{'label': 'display', 'lat': 40.73467444734881...",40.734674,-73.998670,NaN,10011,NY,4f2425cce4b0e12081504088
8,Spring Into Yoga- Central Park,Yoga Studio,102nd St and Fifth Avenue,US,New York,United States,"102nd st, Fifth Ave",567,"[102nd St and Fifth Avenue (102nd st, Fifth Av...","[{'label': 'display', 'lat': 40.79225934919049...",40.792259,-73.953744,NaN,10029,NY,51bd486babd8c63049739fd8
9,Yoga to the People,Yoga Studio,1017 Avenue of the Americas,US,New York,United States,38th St,4743,"[1017 Avenue of the Americas (38th St), New Yo...","[{'label': 'display', 'lat': 40.7523979170956,...",40.752398,-73.986091,NaN,10018,NY,4b4ce47cf964a5203bc326e3


## Visualize the gyms & Yoga centers in Manhattan area

In [33]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Conrad Hotel


# add the gyms as blue circle markers
for lat, lng, label in zip(gym_df_filtered.lat, gym_df_filtered.lng, gym_df_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)
    
    
# add the yoga centers as green circle markers
for lat, lng, label in zip(yoga_df_filtered.lat, yoga_df_filtered.lng, yoga_df_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='green',
        popup=label,
        fill = True,
        fill_color='green',
        fill_opacity=0.6
    ).add_to(venues_map)
    
# add the Manhattan neighborhoods as yellow circle markers
for lat, lng, label in zip(manhattan_data.Latitude, manhattan_data.Longitude, manhattan_data.Neighborhood):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='yellow',
        popup=label,
        fill = True,
        fill_color='yellow',
        fill_opacity=0.6
    ).add_to(venues_map)
# display map
venues_map

In [34]:
gym_df_filtered = gym_df_filtered[['name', 'categories', 'lat', 'lng', 'distance', 'postalCode']]
gym_df_filtered.head()

,name,categories,lat,lng,distance,postalCode
0,Pure Yoga East,Yoga Studio,40.778723,-73.953772,1364,10028
1,Upper West Side Yoga and Wellness,Yoga Studio,40.785682,-73.972310,1160,10024
2,Pure Yoga West,Yoga Studio,40.781659,-73.979545,1908,10024
3,New York Yoga - Hot Studio,Yoga Studio,40.778910,-73.956160,1282,10028
4,Life in Motion: Yoga Cycling Barre,Yoga Studio,40.800515,-73.967910,1345,10025


In [35]:
yoga_df_filtered = yoga_df_filtered[['name', 'categories', 'lat', 'lng', 'distance','address', 'postalCode']]
yoga_df_filtered.head()

,name,categories,lat,lng,distance,address,postalCode
0,Pure Yoga East,Yoga Studio,40.778723,-73.953772,1364,203 E 86th St,10028
1,Upper West Side Yoga and Wellness,Yoga Studio,40.785682,-73.972310,1160,75 W 85th St Apt B,10024
2,Pure Yoga West,Yoga Studio,40.781659,-73.979545,1908,204 W 77th St,10024
3,New York Yoga - Hot Studio,Yoga Studio,40.778910,-73.956160,1282,132 E 85th St,10028
4,Life in Motion: Yoga Cycling Barre,Yoga Studio,40.800515,-73.967910,1345,2744 Broadway,10025


In [36]:
activity_areas = pd.concat([gym_df_filtered, yoga_df_filtered], axis = 0)
activity_areas.drop(['address'], axis = 1, inplace = True)
activity_areas.dropna(axis=0, subset = ['postalCode'], inplace = True)
activity_areas.drop(['categories', 'lat', 'lng', 'name'], axis=1, inplace = True)
activity_areas

,distance,postalCode
0,1364,10028
1,1160,10024
2,1908,10024
3,1282,10028
4,1345,10025
5,1402,10024
6,2313,10027
7,6983,10011
8,567,10029
9,4743,10018


In [37]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=3)
kmeans.fit(activity_areas)


KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=3, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

In [38]:
kmeans.labels_[0:10]

array([0, 0, 0, 0, 0, 0, 0, 2, 0, 2], dtype=int32)

In [ ]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_merged['Latitude'], manhattan_merged['Longitude'], manhattan_merged['Neighborhood'], manhattan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters